In [ ]:
import os
import re
import rdp
import h5py
import numpy as np
import cmocean as cmo
import matplotlib as mpl
from matplotlib import mlab, cm
import matplotlib.mlab as ml
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from scipy.interpolate import griddata
import scipy.ndimage.filters as filters
from scipy.interpolate import RectBivariateSpline
from scipy.ndimage.filters import gaussian_filter
from matplotlib.colors import ListedColormap, BoundaryNorm

# Import the python file (.py) which contains all defined functions
import stratalArchitecture as strata

# Display plots in SVG format
%config InlineBackend.figure_format = 'svg'

# Display plots in cells            
%matplotlib inline

In [ ]:
import plotly
from plotly import tools
import plotly.graph_objs as go
from plotly.graph_objs import *
plotly.offline.init_notebook_mode()

import warnings
warnings.simplefilter(action = "ignore", category = FutureWarning)

In [ ]:
# output folder path
folder = '/live/share/SlabDip_LEM/control_case/h5/' 
strat = []
strat = strata.stratalSection(folder, 1)
# 
timestep = 30
strat.loadStratigraphy(timestep-1)  # load strata files
strat.loadTIN(timestep)  # load TIN files
strat.nz

In [ ]:
strat.plotSectionMap(title='Topography map', xlegend='Distance (m)', ylegend='Distance (m)', 
                     color=cmo.cm.delta, crange=[-2000,2000], cs=None, size=(6,6))

In [ ]:
# Coordinates [x,y] of two points on the cross-section
cs=np.zeros((2,2))
cs[0,:] = [1000000,500000]  # point 1
cs[1,:] = [2000000,500000]  # point 2

# Interpolation parameters
nbpts = 1001  
gfilt = 1  

# Show the location of the cross-section on the topography map
strat.plotSectionMap(title='Topography map', xlegend='Distance (m)', ylegend='Distance (m)',
                     color=cmo.cm.delta, colorcs='magenta', crange=[-2000,2000], cs=cs, size=(6,6))

In [ ]:
# Build cross-section
strat.buildSection(xo = cs[0,0], yo = cs[0,1], xm = cs[1,0], ym = cs[1,1], pts = nbpts, gfilter = gfilt)

In [ ]:
strata.viewSection(width = 900, height = 400, cs = strat, 
            dnlay = 1, rangeX=[0,1e6], rangeY=[-3000,2500],
            linesize = 0.5, title='20degree_uplift5km_500km_flexure_kc5e7_dp09_flex1_30km_highkd_0622')

## Plot the rate of erosion and deposition through time along a cross-section

In [ ]:
def loadTIN(folder = None, timestep = 0, ncpus = 1):
    #
    for i in range(0, ncpus):
        df = h5py.File('%s/tin.time%s.hdf5'%(folder, timestep), 'r')
        coord = np.array((df['/coords']))
        cumthick = np.array((df['/cumdiff']))
        if i == 0:
            x, y, z = np.hsplit(coord, 3)
            cumth = cumthick

    dx = x[1]-x[0]

    nx = int((x.max() - x.min())/dx + 1)
    ny = int((y.max() - y.min())/dx + 1)
    
    xi = np.linspace(x.min(), x.max(), nx)
    yi = np.linspace(y.min(), y.max(), ny)
    xi, yi = np.meshgrid(xi, yi)
    zi = griddata((x[:,0],y[:,0]),z[:,0],(xi,yi),method='nearest')
    cumTh = griddata((x[:,0],y[:,0]),cumth[:,0],(xi,yi),method='nearest')

    return zi, cumTh

In [ ]:
cumTh=[]
zi=[]
timeID = np.arange(0,30)
# timeID = 30
time = timeID*1.0
folder = '/live/share/SlabDip_LEM/control_case/h5/'
for i in timeID:
    temp1, temp2 = loadTIN(folder = folder, timestep = i, ncpus = 1)
    zi.append(temp1)
    cumTh.append(temp2)

In [ ]:
topoWE = []
cumdiffWE = []
cumdiffWE_rate = []
# 
for i in range(len(timeID)):
    topoWE.append(gaussian_filter(zi[i][200:300,551:951], sigma=3))
    cumdiffWE.append(gaussian_filter(cumTh[i][200:300,551:951], sigma=3))
# Average
cumdiffWE_ave = np.zeros((len(topoWE),len(topoWE[0][0])))
topoWE_ave = np.zeros((len(topoWE),len(topoWE[0][0])))
for i in range(len(topoWE)):
    for j in range(len(topoWE[0][0])):
        cumdiffWE_ave[i][j] = sum(cumdiffWE[i][:,j])/100
        topoWE_ave[i][j] = sum(topoWE[i][:,j])/100
# Erosion rate
for i in range(len(timeID)-1):
    cumdiffWE_rate.append(cumdiffWE_ave[i+1]-cumdiffWE_ave[i])

In [ ]:
import matplotlib.colors as mcolors

fig = plt.figure(figsize = (7,3))
plt.rc("font", size=11)
# 
ax = fig.add_axes([0.11, 0.16, 0.87, 0.77])

x = time[1:]
y = np.arange(0, topoWE[0].shape[1]*2, 2)
X, Y = np.meshgrid(y, x)
colors1 = plt.cm.pink_r(np.linspace(0., 1, 41))
colors2 = plt.cm.cubehelix(np.linspace(0, 1, 215))

# combine them and build a new colormap
colors = np.vstack((colors1, colors2))
mymap = mcolors.LinearSegmentedColormap.from_list('my_colormap', colors)

norm1 = cm.colors.Normalize(vmax=500, vmin=0)
levels1 = np.arange(0, 500, 5)
cset1 = ax.contourf(X,Y,cumdiffWE_rate, levels1,
                     cmap=cm.get_cmap(mymap, len(levels1) - 1), norm=norm1)
plt.colorbar(cset1)
cset2 = plt.contour(cset1, levels=cset1.levels[::20],
                  colors='k',linewidths=(0.7,),
                  origin = 'lower')

# fig.savefig("EroDepRateMap.pdf", dpi=400)

## Visualize stratal stacking pattern coloured by facies

First we build paleo-depositional environment (facies) structure based on the paleo-water depth. For example ([reference](https://opentextbc.ca/geology/chapter/6-3-depositional-environments-and-sedimentary-basins/)),

<img src="depo-envi.png" alt="depositional environments" width="800" height="500"/>

In [ ]:
strat_secDep = np.array((strat.secDep))
strat_secElev = np.array((strat.secElev))
strat_secTh = np.array((strat.secTh))
strat_nz = strat.nz

In [ ]:
def buildEnviID_facies(secElev = None, secTh = None, nz = None, depthIDs = None):
    envi = np.zeros((nz, len(depthIDs)+2))
    
    for i in range(0,nz):
        onlapID = np.where(secElev[i] <= 300)[0]
        if(len(onlapID)>0):
            envi[i][0] = np.amin(onlapID) # onlap
        downlapID = np.where((secTh[i]) >= 1.5)[0]
        if(len(downlapID)>0):
            envi[i][len(depthIDs)+1] = np.amax(downlapID)
            
        for j in range(len(depthIDs)):
            rangeID = np.where((secElev[i]) <= depthIDs[j])[0]
            if(len(rangeID)==0):
                envi[i][j+1] = max(envi[i][len(depthIDs)+1],envi[i][j])
            else:
                envi[i][j+1] = np.amin(rangeID)
    #
    return envi

In [ ]:
# Specify the range of water depth for the depositional environments, see the table above
depthID = [0, -30, -100, -300, -500]
# depthID = [0, -20, -50, -150, -300, -500]

# Build an array of depositional environment ID (enviID)
enviID = np.zeros((strat.nz-1, len(depthID)+2))
enviID = buildEnviID_facies(secElev = strat_secElev, secTh = strat_secTh, nz = strat_nz-1, depthIDs = depthID)

In [ ]:
# Plot stratal stacking pattern colored by paleo-depositional environments
fig = plt.figure(figsize = (7,3))
plt.rc("font", size=11)
# 
ax = fig.add_axes([0.13, 0.16, 0.82, 0.79])
# p = 0
# layID = []
xi00 = np.array(strat.dist)/1000.  # change the unit to be be km
colors = ['limegreen','darkkhaki','sandybrown','khaki','c','teal']
# colors = ['khaki','mediumseagreen','skyblue','lightgray','darkkhaki','c','teal']
for i in range(0,strat_nz-1,1):
#     if i == strat_nz-1:
#         i=strat_nz-1
#     layID.append(i)
    ax.plot(xi00,strat_secDep[i],'-',color='k',linewidth=0.1)
#     if len(layID)>1:
    for j in range(len(depthID)+1):
        ID1=enviID[i][j]
        ID2=enviID[i][j+1]
        if (ID1<ID2):
            for k in range(int(ID1),int(ID2)):
                ax.fill_between([xi00[k],xi00[k+1]], [strat_secDep[i][k], strat_secDep[i][k+1]],
                            max(strat_secDep[i+1]), color=colors[j])
    if (max(strat_secDep[i+1]) <= max(strat_secDep[i])):
        ax.fill_between(xi00, strat_secDep[i+1], max(strat_secDep[i]), color='white')
    else:
        ax.fill_between(xi00, strat_secDep[i+1], max(strat_secDep[i+1]), color='white')
#     p=p+1
#   
strat_max = np.maximum(strat_secDep[strat_nz-1], 0)
ax.fill_between(xi00, strat_secDep[strat_nz-1], strat_max, color='lightblue', alpha=0.5)
ax.plot(xi00,strat_secDep[strat_nz-1],'-',color='k',linewidth=0.3)  # top line
ax.plot(xi00,strat_secDep[0],'-',color='k',linewidth=0.5)  # bottom line
ax.plot(xi00,strat_secDep[10],'-',color='k',linewidth=0.5)  
ax.plot(xi00,strat_secDep[20],'-',color='k',linewidth=0.5)  
ax.plot(xi00,strat_secDep[30],'-',color='k',linewidth=0.5)  
ax.set_xlim([100, 900])
ax.set_ylim([-2000, 500])
ax.set_xlabel('Distance (km)',fontsize=11)
ax.set_ylabel('Elevation (m)',fontsize=11)

# Save the plot
# fig.savefig("Strata.pdf", dpi=300)
plt.close(fig)

## Calculate the proportion of each depositional facies

In [ ]:
alluvial_plain = []
shoreface = []
distal_offshore = []
upper_slope = []
middle_slope = []
abyssal = []
for j in range(strat_nz-1):
    alluvial_plain.append(sum(strat.secTh[j][int(enviID[j][0]):int(enviID[j][1])]))
    shoreface.append(sum(strat.secTh[j][int(enviID[j][1]):int(enviID[j][2])]))
    distal_offshore.append(sum(strat.secTh[j][int(enviID[j][2]):int(enviID[j][3])]))
    upper_slope.append(sum(strat.secTh[j][int(enviID[j][3]):int(enviID[j][4])]))
    middle_slope.append(sum(strat.secTh[j][int(enviID[j][4]):int(enviID[j][5])]))
    abyssal.append(sum(strat.secTh[j][int(enviID[j][5]):int(enviID[j][6])]))

In [ ]:
# Percentage of each facies
alluvial_plain_perc = sum(alluvial_plain)/sum(sum(strat.secTh[:]))*100
shoreface_perc = sum(shoreface)/sum(sum(strat.secTh[:]))*100
distal_offshore_perc = sum(distal_offshore)/sum(sum(strat.secTh[:]))*100
upper_slope_perc = sum(upper_slope)/sum(sum(strat.secTh[:]))*100
middle_slope_perc = sum(middle_slope)/sum(sum(strat.secTh[:]))*100
abyssal_perc = sum(abyssal)/sum(sum(strat.secTh[:]))*100

facies_perc = np.array([alluvial_plain_perc, shoreface_perc, distal_offshore_perc, upper_slope_perc, 
                        middle_slope_perc, abyssal_perc])

In [ ]:
fig = plt.figure(figsize = (6,3))
plt.rc("font", size=12)
# 
ax = fig.add_axes([0.13, 0.15, 0.82, 0.75])
facies = ['alluvial_plain', 'shoreface', 'distal_offshore', 'upper_slope', 'middle_slope', 'abyssal']
bar = ax.bar(facies, facies_perc, width = 0.5, color=['limegreen','darkkhaki','sandybrown','khaki','c','teal'])
# ax.set_ylim([0, 90])
ax.set_xlabel('Depositional facies',fontsize=11)
ax.set_ylabel('Percentage (%)',fontsize=12)
ax.set_xticks('off')
ax.set_xticklabels(facies, rotation=-15)
ax.grid(color='lightgray', linestyle='-', linewidth=0.5)

# Save the plot
# fig.savefig("FaciesPerc.pdf", dpi=300)

## Build a Wheeler diagram

Wheeler diagram (or chronostratigraphic chart) is a powerful tool to document unconformities between sequences, and to understand the evolution of sedimentary stacking patterns and their relationships to sea level. It displays the horizontal distribution of contemporaneous sedimentary layer sequences, as well as hiatuses in sedimentation.

In [ ]:
def buildEnviID_WD(secElev = None, secTh = None, nz = None, dist = None, depthID = None):
    enviID = np.zeros((nz, len(dist)))
    
    # Build a 2D array of the depositional environment ID (enviID) 
    for j in range(nz):
        for i in range(len(dist)):
            if (secElev[j][i]) > (depthID[0]):
                enviID[j][i] = 0
            elif (secElev[j][i]) > (depthID[1]):
                enviID[j][i] = 1
            elif (secElev[j][i]) > (depthID[2]):
                enviID[j][i] = 2
            elif (secElev[j][i]) > (depthID[3]):
                enviID[j][i] = 3
            elif (secElev[j][i]) > (depthID[4]):
                enviID[j][i] = 4
            else:
                enviID[j][i] = 5
    
    # Where the deposited thickness is less than 0.5 m, the enviID will be set to -1 (i.e. will be coloured in white).
    # You can change the value of '0.5'.
    for j in range(nz):
        for i in range(len(dist)):
            if (secTh[j][i]) <= 0.5:
                enviID[j][i] = -1
    
    return enviID

In [ ]:
# Define colors for depositional environments, with number of colors equals to len(depthID) + 2
colorDepoenvi = ['white','limegreen','darkkhaki','sandybrown','khaki','c','teal'] 
# 'White' colors where either no deposition or deposited sediemnt thickness < 0.5 m.

# Build an array of depositional environment ID (enviID_WD)
enviID_WD = np.zeros((strat_nz, len(strat.dist)))
enviID_WD = buildEnviID_WD(secElev = strat_secElev,secTh = strat_secTh,nz = strat_nz-1,dist = strat.dist,depthID = depthID)

# Time structure of the model, corresponding to the Time structure in the input.xml file
start_time = 0.  # the start time of the model run [a]
disptime = 1000000.  # the layer interval of the strata module [a]
end_time = start_time + disptime * timestep  # the time of the loaded output [a]
layertime = np.linspace(start_time,end_time,strat_nz)  # time of the layers

In [ ]:
# Plot Wheeler diagram
fig = plt.figure(figsize = (7,3))
plt.rc("font", size=11)
    
ax = fig.add_axes([0.13, 0.16, 0.82, 0.79])
# Define the meshgrid of the Wheeler diagram, in which X axis is distance along the cross-section and Y axis is time
dist = strat.dist  
Dist, Time = np.meshgrid(dist, layertime/1e6)

cmap = ListedColormap(colorDepoenvi)
boundaries = [-1, 6]
levels=[-1.5,-0.5,0.5,1.5, 2.5, 3.5, 4.5, 5.5]
# norm = colors.Normalize(boundaries, cmap.N)
norm = cm.colors.Normalize(vmax=6, vmin=-1)
ax.imshow(enviID_WD[:,:], interpolation='nearest', cmap=cmap, origin='lower', 
           extent=[dist.min()/1000, dist.max()/1000, (layertime/1e6).min(), (layertime/1e6).max()], 
           aspect='auto', vmax=5.5, vmin=-1.5)
# plt.colorbar()

# Plot the horizontal time lines with the same interval with the stratal stacking pattern
for i in range(0,strat_nz,1): 
    ax.axhline(layertime[i]/1e6, color='lightgray', linewidth=0.1)

ax.set_xlim([100,900])
ax.set_ylim([0.,30])
ax.set_xlabel('Distance (km)')
ax.set_ylabel('Time (Myr)')
# 
# fig.savefig("WheeDiag.pdf", dpi=300)